In [7]:
import spacy
import nltk
import import_ipynb
import frame
import nlg
from frame import Frame
import dependency
import natural_language_understanding as nlu
from nlu import NLU
import padawian_list
import random
from constraint import *
import categories

In [1]:
class DialogManager: 
    def __init__(self):
        self.category = None
        self.feature_target = None
        self.questions = None
        self.score = 0 
        self.personaggio = random.choice(PERSONAGGIO)
        self.nlg = nlg.GenerateText()

    
    def get_category(self):
        cat = random.choice(categories.categories)
        while (self.category == cat):
             cat = random.choice(categories.categories)
        self.category = cat #si assegna la categoria random
        self.feature_target = self.category.get_features() #prendiamo le feature della categoria
        self.questions = self.category.get_questions() #lista delle domande
        

#DOMANDE --> Bisogna migliorare qualche cosa. 

    #Genero l'introduzione a partire dalla frase generate da nlg. Frase del tipo "Hi Youngligs, Let's talk about Jedi Council"
    def introduction(self):
        intro = self.nlg.introduzione()
        print(intro)
        #self.speak(intro)

    #Faccio partire la fase preliminare delle domande. Del tipo "Well Youngling, let's start the exam"
    def pre_questions(self):
        pre_q = self.nlg.start_questions()
        print(pre_q)


    #Faccio una domanda random a partire dalla lista di domande generate dalla lista di domande della categoria selezionata
    def ask_question(self):
        
        question = random.choice(self.questions) #prendo una domanda random
        while(question.check_asked() == True):
            return self.ask_question()
            
        print(f"{self.personaggio}: {question.get_question()}")
        question.set_asked() #setto la domanda come gia fatta
        return question #la stampo

    #Definisco un metodo che mi stampa l'interazione utente (Padawan:  )
    def padawan_interaction(self,question):
        answer = input("Youngling: ")
        question.set_user_answer(answer)
        self.check_answer(answer, self.category.get_features(), question)
        return question


    #Ricevo la risposta e la analizzo
    def check_answer(self, padawan_answer, target_features, question):
        nlu_answer = nlu.NLU(padawan_answer)
        sim = nlu_answer.check_response(padawan_answer, [question.get_answer()])
        find_feature = nlu_answer.check_features(target_features, question)
        sim_idk = nlu_answer.check_response(padawan_answer,UNCERTAIN_RESPONSE)
        #se il target e' quello giusto. Ovvero se la risposta data e' positiva e/o negativa e fa riferimento al target
        if sim_idk >= 0.6:
            answer = self.nlg.generate_idk_answer()
            print(f"{self.personaggio}:  {answer}")
            self.score -= 0.5 #vedere penalità del non lo so 
            
        else: 
        #Resituisco il feed alla risposta
            if sim >= 0.7:
                answer = self.nlg.generate_good_answer()
                print(f"{self.personaggio}:  {answer}")
                self.score += 1 #inventarci un calcolo del punteggio che si basa sulla similarità
            else:
                answer = self.nlg.generate_negative_answer()
                print(f"{self.personaggio}:  {answer}")
                self.score -= 1 #inventarci un calcolo del punteggio che si basa sulla similarità
        


    #def check_padawian_answers(self, answer, question):

    #Controllo se una domanda presenta una delle caratteristiche target 
    """
    def check_questions(self, question):
        target = dependency.Dependency(question)
        features = target.check_dependency_padawian()

        for f in features:
            if f in self.feature_target:
                return True
            else:
                return False
    """

    #Controllo se ci sono eventuali duplicati nella domanda. Ovvero controllo seu una delle caratteristiche dette e' gia presente in memoria o no
    def check_duplicate_features(self, response):
        q = self.questions.generate_text(response)
        features = dependency.Dependency(q)

        for f in features.check_dependency_padawian():
            if f in self.memory:
                return self.check_duplicate()
            else:
                self.memory.append(f.check_dependency_padawian())
                return q



#RISPOSTE --> E' da migliorare   


        
    #Controllo il tipo di risposta data presenta la feature target
    def check_response_target(self, answer):
        target = dependency.Dependency(answer)
        features = target.check_dependency_padawian()

        #praticamente se la risposta data e' nulla
        if len(features)==0:
            self.wrong_answer += 1
            answer = self.bad_answers
            print(f"{self.personaggio}:  {answer}")

        #se la risposta data e' positiva e fa riferimento al target
        elif features[0] in self.feature_target:
            self.frame.add_feature(self.memory[-1])
            answer = self.good_answers
            print(f"{self.personaggio}:  {answer}")
            self.memory.append(features)

        #se la risposta data e' negativa e fa riferimento al target
        elif features[0] not in self.feature_target:
            self.wrong_answer += 1
            answer = self.bad_answers
            print(f"{self.personaggio}:  {answer}")
            self.memory.append(features)
        
        #se la risposta data e' un duplicato
        elif self.check_duplicate_features(features[0]):
            self.wrong_answer += 1 
            neg_response = ' Youngling, this has already been mentioned'
            print(f"{self.personaggio}: {neg_response}")

    #definisco un commento che da una votazione al padawian
    def comments_obi_wan(self):
        vote = 30 - self.wrong_answer #ogni volta che si risponde male a una domanda viene sottratto un punto al voto

        if vote >= 28 and vote <= 30:
            comment = 'Excellent, you are ready to become a Padawian'
        
        elif vote >= 25 and vote <= 27:
            comment = 'Very good, you are almost ready to become a Padawian'

        elif vote >= 20 and vote <= 24:
            comment = 'Good job but you need to study more to become Padawian'
        
        elif vote >= 18 and vote <= 20:
            comment = 'You need to study more to become Padawian'

        return comment
    
    #con Darth Fener il voto viene penalizzato
    def comments_darth_fener(self):
        penalty = random.randint([1.5, 2.5, 3.5])
        vote = 30 - (self.wrong_answer*penalty)
        if vote >= 28 and vote <= 30:
            comment = 'Unfortunatly, you are ready to become a Padawian'
        
        elif vote >= 25 and vote <= 27:
            comment = 'What a pitty, you are almost ready to become a Padawian'

        elif vote >= 20 and vote <= 24:
            comment = 'Good job but you need to study more to become Padawian'
        
        #penalita' maggiore con darth Fener
        elif vote >= 18 and vote <= 20:
            return self.failure()
            
        return comment

    
    def failure_obi_wan(self):
        comment = "You cannot become a Padawian. You need to study more."
        return comment
    
    def failure_darth_fener(self):
        comment = "You are a foul, you will never become a Padawan. Join to the dark side."

        return comment


    #definisco il metodo per il voto
    def grade_obi_wan(self):
        grade = self.comments_obi_wan()
        print(f"\t {grade}")

    def grade_darth_fener(self):
        grade = self.comments_darth_fener()
        print(f"\t {grade}")

#NLG
    #Faccio partire l'esame
    def start_exam(self):

        self.introduction()

        self.get_category()
        self.nlg.set_category(self.category)
        self.nlg.set_character(self.personaggio)
        self.pre_questions()
        
        question = self.ask_question() #domanda fatta salvata
        question=self.padawian_interaction(question) #prende la risposta del padawan e la salva


        if self.personaggio == 'Obi Wan Kenobi':

                if self.wrong_answer == 4:
                    return self.failure_obi_wan()
            
                return self.grade_obi_wan()
        
        if self.personaggio == 'Darth Fener':
           
            while not self.frame.check_finish():
                question = self.ask_question()
                self.padawian_interaction(question)

                if self.wrong_answer == 4:
                    return self.failure_darth_fener()
        
            return self.grade_darth_fener()

In [ ]:
if __name__ == "__main__":
    exam = DialogManager()
    exam.start_exam()